# Building an Anime Recommender System

In [1]:
import sys
sys.path.append("..")

from animerec import models
from animerec import utils

import pandas
import pickle

### 1. Load Dataset and Model

In [2]:
# Load anime metadata.
pickle_in = open("../data/animes.pickle", 'rb')
animes = pickle.load(pickle_in)
animes.index = animes['anime_id']
animes = animes[ ['title', 'title_english', 'type', 'premiered', 'genre'] ]
pickle_in.close()

# Load model trained on server.
pickle_in = open("../model/model.pickle", 'rb')
model = pickle.load(pickle_in)
pickle_in.close()

# Load anime id's.
pickle_in = open("../model/item-id.pickle", 'rb')
item_id = pickle.load(pickle_in)
pickle_in.close()

### 3. Webscrape current profile and create prediction

In [18]:
myprofile = utils.get_user_anime_list('Manuel')
x = utils.get_score_vector_from_user_anime_list(myprofile, item_id)

Scraped 300 additional anime from https://myanimelist.net/animelist/Manuel/load.json?offset=0&status=7
Scraped 175 additional anime from https://myanimelist.net/animelist/Manuel/load.json?offset=300&status=7


In [19]:
rhat = model.predict(x)

In [20]:
prediction = utils.prediction_to_dataframe(rhat, myprofile, item_id, keep_all=False)
prediction = pandas.DataFrame(prediction)
prediction.columns = ['score']
prediction = prediction.join(animes)

In [21]:
prediction.sort_values(by='score', ascending=False).head(20)

,score,title,title_english,type,premiered,genre
721,9.581140,Princess Tutu,Princess Tutu,TV,Summer 2002,"Comedy, Drama, Magic, Romance, Fantasy"
3297,9.428207,Aria The Origination,Aria The Origination,TV,Winter 2008,"Fantasy, Sci-Fi, Shounen, Slice of Life"
17074,9.378211,Monogatari Series: Second Season,Monogatari Series: Second Season,TV,Summer 2013,"Mystery, Comedy, Supernatural, Romance, Vampire"
35247,9.317616,Owarimonogatari 2nd Season,Owarimonogatari Second Season,TV,Summer 2017,"Comedy, Mystery, Supernatural, Vampire"
1453,9.296081,Maison Ikkoku,Maison Ikkoku,TV,Spring 1986,"Slice of Life, Comedy, Drama, Romance, Seinen"
6594,9.240585,Katanagatari,Katanagatari,TV,Winter 2010,"Action, Adventure, Historical, Martial Arts, R..."
2921,9.225994,Ashita no Joe 2,Rocky Joe 2,TV,Fall 1980,"Action, Drama, Shounen, Slice of Life, Sports"
44,9.171223,Rurouni Kenshin: Meiji Kenkaku Romantan - Tsui...,Samurai X: Trust and Betrayal,OVA,NaN,"Action, Historical, Drama, Romance, Martial Ar..."
66,9.125869,Azumanga Daioh,Azumanga Daioh: The Animation,TV,Spring 2002,"Slice of Life, Comedy, School"
31758,9.097629,Kizumonogatari III: Reiketsu-hen,NaN,Movie,NaN,"Action, Mystery, Supernatural, Vampire"


In [7]:
users = pandas.read_csv("../data/raw/UserList.csv")
idx = users.index[users.username == 'Manuel'][0]

In [11]:
rhat = model.U[idx].dot(model.V.T) + model.bu[idx] + model.bv + model.mu

In [12]:
prediction = utils.prediction_to_dataframe(rhat, myprofile, item_id, keep_all=False)
prediction = pandas.DataFrame(prediction)
prediction.columns = ['score']
prediction = prediction.join(animes)

In [13]:
prediction.sort_values(by='score', ascending=False).head(20)

,score,title,title_english,type,premiered,genre
32995,10.715361,Yuri!!! on Ice,Yuri!!! On ICE,TV,Fall 2016,"Comedy, Sports"
10161,10.666838,No.6,No. 6,TV,Summer 2011,"Action, Sci-Fi, Mystery, Drama"
33352,10.570225,Violet Evergarden,Violet Evergarden,TV,Winter 2018,"Slice of Life, Drama, Fantasy"
35062,10.508804,Mahoutsukai no Yome,The Ancient Magus&#039; Bride,TV,Fall 2017,"Slice of Life, Fantasy, Magic, Shounen"
34240,10.330217,Shelter,Shelter,Music,NaN,"Sci-Fi, Music"
10321,10.065937,Uta no☆Prince-sama♪ Maji Love 1000%,Uta no Prince Sama,TV,Summer 2011,"Harem, Music, Comedy, Romance, School, Shoujo"
431,10.022015,Howl no Ugoku Shiro,Howl&#039;s Moving Castle,Movie,NaN,"Adventure, Drama, Fantasy, Romance"
32729,10.007061,Orange,Orange,TV,Summer 2016,"Sci-Fi, Drama, Romance, School, Shoujo"
22265,9.996372,Free!: Eternal Summer,Free! - Eternal Summer,TV,Summer 2014,"Slice of Life, Comedy, Sports, Drama, School"
731,9.979772,Interstella5555: The 5tory of The 5ecret 5tar ...,NaN,Music,NaN,"Adventure, Drama, Music, Sci-Fi"
